<a href="https://colab.research.google.com/github/cmlins/recuperacaoDeInformacao/blob/master/wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests as rq
import re

### Bulas

Request da página e parser usando BeautifulSoup

In [0]:
# url = "https://consultaremedios.com.br/buscoduo/p"
# url = 'https://consultaremedios.com.br/neocopan-composto-comprimido/p'
# url = 'https://consultaremedios.com.br/cefaliv/p'
url = 'https://consultaremedios.com.br/doril-enxaqueca/p'

page = rq.get(url)
# print(page.text)

soup = bs(page.text,"html.parser")
# print(soup.prettify())

Coleta de informações sobre o medicamento e criação do dataframe com as informações

In [0]:
name_div = soup.find('div', {'class':'product-header-wrapper'})

name = name_div.h1.text
# print(name)

info_col = []
info_res = []

data = soup.find_all("div", {"class": "extra-infos-block"})

for info in data:
  info_col.append(info.h3.contents[0])
  info_res.append(info.text)
  
for i in range(len(info_res)):
  spl = info_col[i]
  info_res[i] = (info_res[i].split(spl)[1])
  
info_col.insert(0, 'Medicamento')
info_res.insert(0, name)

# print(info_col)
# print(info_res)

list_drug=[]

list_drug.append(dict(zip(info_col,\
                          info_res)))

pd_drug = pd.DataFrame(list_drug,\
                       columns=info_col)

In [4]:
pd_drug

,Medicamento,Fabricante,Tipo do Medicamento,Necessita de Receita,Princípio Ativo,Categoria do Medicamento,Classe Terapêutica,Especialidades
0,Doril Enxaqueca,Cosmed,Medicamento Similar,"Não, Isento de Prescrição Médica",Ácido Acetilsalicílico + Cafeína + Paracetamol,Dor de Cabeça e Enxaqueca,Todos As Outras Preparações Antienxaquecosas,"Ginecologia, Otorrinolaringologia"


#### Página com vários medicamentos

In [0]:
url = 'https://consultaremedios.com.br/aparelho-respiratorio/rinite-e-sinusite/c'

page = rq.get(url)
# print(page.text)

soup = bs(page.text,"html.parser")
# print(soup.prettify())


In [0]:
drugs_name = soup.find_all('span', {'class': 'product-block__product-name'})

drug_list = []

for drug in drugs_name:
  drug_list.append(drug.text)

# len(drug_list)

In [0]:
active_ingredient = soup.find_all('a', {'class':'product-block__substance product-block__meta-info'})

active_ingredient_list = []

for ing in active_ingredient:
  active_ingredient_list.append(ing.text)
  
# len(active_ingredient_list)

In [0]:
labs = soup.find_all('a', {'class':'product-block__factory product-block__meta-info'})

labs_list = []

for lab in labs:
  labs_list.append(lab.text)
  
# len(labs_list)

In [0]:
prices = soup.find_all('div', {'class':'product-block__starting-price-value'})

price_list = []

for price in prices:
  price_list.append(price.text)
  
# len(price_list)

Está errado abaixo --> item 4, o laboratório deveria ser BOIRON

In [0]:
cols = ['Nome', 'Princípio ativo', 'Laboratório', 'Preço']

list_drugs = zip(drug_list,active_ingredient_list,labs_list,price_list)

drugs_pd = pd.DataFrame(list_drugs,\
                       columns=cols)

In [11]:
drugs_pd

,Nome,Princípio ativo,Laboratório,Preço
0,Actifedrin,Pseudoefedrina + Triprolidina,Farmoquímica,"R$ 9,07"
1,Avamys,Furoato de Fluticasona,GSK,"R$ 39,87"
2,Aznite,Cloridrato de Azelastina,EMS,"R$ 24,13"
3,Complexo Homeopático Almeida Prado Nº 3,Hydrastis canadensis + Magnesia carbonica + Ka...,Homeopatia Almeida Prado,"R$ 5,01"
4,Coryzalia,Allium Cepa 3CH + Associações,Neo Química,"R$ 33,90"
5,Cromoglicato de Sódio Neo Química,Cromoglicato Dissódico,Allergan,"R$ 7,96"
6,Cromolerg,Cromoglicato Dissódico,Meda Pharma,"R$ 9,27"
7,Dymista,Propionato de Fluticasona + Cloridrato de Azel...,GSK,"R$ 86,80"
8,Flixonase,Propionato de Fluticasona,GSK,"R$ 43,21"
9,Flixotide Diskus,Propionato de Fluticasona,GSK,"R$ 83,55"


# Sites Livros

1.   https://www.amazon.com.br/
2.   https://www.submarino.com.br/
3.   https://www.saraiva.com.br/
4.   https://www.estantevirtual.com.br/
5.   https://www.americanas.com.br/
6.   https://www.ciadoslivros.com.br/
7.   https://www.traca.com.br/
8.   https://www.livrariacultura.com.br
9.   https://www.disal.com.br/
10.  https://www.extra.com.br/livros/

##AMAZON

### All books in a page

In [0]:
url = 'https://www.amazon.com.br/Livros/b?ie=UTF8&node=6740748011&ref_=nav_books_all'

page = rq.get(url)
# print(page.text)

soup = bs(page.text,"html.parser")

In [0]:
books = soup.find_all('li', {'class':'a-carousel-card acswidget-carousel__card'})

In [0]:
books_name = []
for book in books:
  books_name.append(book.span.contents)
# books_name

for book in books:
  print(book.div.contents)

In [0]:
books_list = []

for book in books:
  book = re.sub("\n|\t|\xa0|[R$]", " ", (book.text.strip()))
  book = re.sub("[,]", '.', book)
  books_list.append(book)
  
for i in range(len(books_list)):
  books_list[i] = books_list[i].split('    ')
  books_list[i] = list(filter(None, books_list[i]))
  books_list[i] = [item.strip() for item in books_list[i]]
  if (len(books_list[i]) == 6):
    books_list[i] = [books_list[i][0],books_list[i][1],books_list[i][2],books_list[i][4]]



In [0]:
books_list

In [0]:
col = ['Título', 'Autor', 'Acabamento', 'Preço']

books_pd = pd.DataFrame(books_list,\
                       columns=col)

In [88]:
books_pd

,Título,Autor,Acabamento,Preço
0,Do Mil ao Milhão. Sem Cortar o...,Thiago Nigro,Capa comum,34.90
1,O milagre da manhã,Hal Elrod. Marcelo Schild,Capa comum,39.90
2,A Sutil Arte de Ligar o F*da-Se: Uma...,Mark Manson. Joana Faro,Capa comum,34.90
3,Mais esperto que o diabo: O mistério...,Napoleon Hill,Capa comum,42.90
4,Os segredos da mente milionária,T. Harv Eker. Pedro Jorgensen Junior,Capa comum,34.90
5,Pai rico. pai pobre 20 anos,obert Kiyosaki,Capa comum,79.90
6,Bíblia King James Atualizada,King James,Imitação de couro,167
7,O poder da ação,Paulo Vieira,Capa comum,34.90
8,Como Fazer Amigos e Influenciar Pessoas,Dale Carnegie,Capa comum,69.90
9,Me Poupe!: 10 passos para nunca mais...,Nathalia Arcuri,Capa comum,34.90


### Information from a single book

In [0]:
# url = 'https://www.amazon.com.br/gp/product/855100249X/ref=s9_acsd_al_bw_c_x_3_w?pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-5&pf_rd_r=W2SAJXDVWZY3MG36EPGB&pf_rd_t=101&pf_rd_p=8cf7d30a-a852-4221-92d0-7c86785d475d&pf_rd_i=6740748011'

# page = rq.get(url)
# # print(page.text)

# soup = bs(page.text,"html.parser")

url = r'/content/A Sutil Arte de Ligar o F_da-Se_ Uma estratégia inusitada para uma vida melhor - 9788551002490 - Livros na Amazon Brasil.html'
soup = bs(open(url),"html.parser")

In [0]:
# print(soup.prettify())

In [0]:
title = (soup.find('span', {'id':'productTitle'})).text
# print(title)

author = (soup.find('span', {'class':'author notFaded'})).find('a',{'class':'a-link-normal'}).text#.contents[0]
# print(author)

book = soup.find('div', {'class':'content'}).find_all('li')[:5]
# for info in book:
#   print(info.text)

n_pages = book[0].contents[1].strip().split(' ')[0]
# print(n_pages)

publishing = book[1].contents[1].strip().split(';')[0]
# print(publishing)

edition = book[1].contents[1].strip().split(' ')[2]
# print(edition)

language = book[2].contents[1].strip()
# print(language)


In [0]:
cols = ['Título', 'Autor', 'Nº Páginas', 'Editora', 'Edição', 'Idioma']

list_info = [title, author, n_pages, publishing, edition, language]
list_info
book_pd = pd.DataFrame([list_info],\
                       columns=cols)


In [22]:
book_pd

,Título,Autor,Nº Páginas,Editora,Edição,Idioma
0,A Sutil Arte de Ligar o F*da-Se: Uma estratégi...,Mark Manson,224,Intrínseca,1ª,Português
